# Reassign Session IDs and Recalculate Summary_Sessions

This notebook:
1. Groups records by (user_id, test_date, boyfriend_name) combination
2. Assigns the same session_id to all records with matching combinations across all CSV files
3. Recalculates Summary_Sessions based on the updated data

**Use this when:**
- Session IDs are inconsistent across tables
- You need to fix session ID assignments based on user_id, test_date, and boyfriend_name
- You want to recalculate Summary_Sessions from scratch


In [ ]:
import sys
from pathlib import Path
import pandas as pd
from decimal import Decimal
from datetime import datetime
import hashlib

# Add project root to path
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))

from src.utils.session_id_generator import generate_session_id
from src.utils.constants import DATE_FORMAT, CSV_SEPARATOR

# Configuration
DATA_DIR = project_root / "data"
CSV_FILES = {
    "session_responses": "session_responses.csv",
    "session_gtk_responses": "session_gtk_responses.csv",
    "session_feedback": "session_feedback.csv",
    "session_toxicity_rating": "session_toxicity_rating.csv",
    "session_insights": "session_insights.csv",
}


In [ ]:
def generate_session_id_with_date(user_id: str, boyfriend_name: str, test_date: str) -> int:
    """
    Generate a unique, deterministic session_id based on user_id, boyfriend_name, and test_date.
    
    Args:
        user_id: Unique identifier for the user
        boyfriend_name: Name of the boyfriend being rated
        test_date: Test date (session date)
        
    Returns:
        A positive integer session_id
    """
    # Normalize inputs
    user_id_norm = str(user_id).lower().strip()
    bf_name_norm = str(boyfriend_name).lower().strip()
    test_date_norm = str(test_date).strip()
    
    # Create a deterministic string from all three components
    combined = f"{user_id_norm}_{bf_name_norm}_{test_date_norm}"
    
    # Generate hash using SHA256
    hash_obj = hashlib.sha256(combined.encode('utf-8'))
    hash_hex = hash_obj.hexdigest()
    
    # Convert first 8 characters of hash to integer
    session_id = int(hash_hex[:8], 16) % (2**31 - 1)
    
    # Ensure it's positive and at least 1
    if session_id == 0:
        session_id = 1
    
    return session_id


In [ ]:
def get_test_date_column(df: pd.DataFrame) -> str:
    """Determine which column contains the test date."""
    # Check common column names for test date (in priority order)
    # Prefer session_start_time as it's more consistent across tables
    date_columns = ['session_start_time', 'test_date', 'timestamp']
    for col in date_columns:
        if col in df.columns:
            return col
    return None

def normalize_date(date_value):
    """Normalize date to string format using consistent DATE_FORMAT."""
    if pd.isna(date_value):
        return None
    # Convert to string and strip
    date_str = str(date_value).strip()
    # If it's a datetime object, format it consistently
    try:
        if isinstance(date_value, pd.Timestamp):
            return date_value.strftime(DATE_FORMAT)
        # Try parsing if it's a string and reformat
        parsed_date = pd.to_datetime(date_str)
        return parsed_date.strftime(DATE_FORMAT)
    except:
        return date_str


In [ ]:
def detect_csv_separator(file_path):
    """Detect CSV separator by reading first line."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            first_line = f.readline()
            # Count semicolons and commas
            semicolon_count = first_line.count(';')
            comma_count = first_line.count(',')
            # Use semicolon if it appears, otherwise comma
            if semicolon_count > 0:
                return ';'
            elif comma_count > 0:
                return ','
            else:
                return CSV_SEPARATOR  # Default
    except:
        return CSV_SEPARATOR  # Default fallback

def load_and_prepare_data():
    """Load all CSV files and prepare them for session ID assignment."""
    print("=" * 60)
    print("Loading CSV Files")
    print("=" * 60)
    
    data = {}
    
    for table_name, filename in CSV_FILES.items():
        file_path = DATA_DIR / filename
        if not file_path.exists():
            print(f"[WARNING] File not found: {filename}")
            continue
        
        try:
            # Detect separator for each file
            separator = detect_csv_separator(file_path)
            df = pd.read_csv(file_path, sep=separator)
            print(f"[OK] Loaded {filename}: {len(df)} records (separator: '{separator}')")
            data[table_name] = df
        except Exception as e:
            print(f"[ERROR] Failed to load {filename}: {e}")
    
    return data


In [ ]:
def create_session_mapping(data: dict) -> dict:
    """
    Create a mapping of (user_id, test_date, boyfriend_name) -> session_id.
    
    Returns:
        Dictionary mapping (user_id, test_date, boyfriend_name) tuples to session_id
    """
    print("\n" + "=" * 60)
    print("Creating Session ID Mapping")
    print("=" * 60)
    
    session_mapping = {}
    
    # Process each table to extract unique combinations
    for table_name, df in data.items():
        if df.empty:
            continue
        
        # Get required columns - check with case-insensitive matching
        user_id_col = None
        boyfriend_name_col = None
        
        for col in df.columns:
            col_lower = col.lower().strip()
            if col_lower == 'user_id':
                user_id_col = col
            elif col_lower == 'boyfriend_name':
                boyfriend_name_col = col
        
        if not user_id_col or not boyfriend_name_col:
            print(f"[WARNING] {table_name} missing required columns. Available columns: {list(df.columns[:10])}")
            continue
        
        # Get test_date column
        date_col = get_test_date_column(df)
        if not date_col:
            print(f"[WARNING] {table_name} has no date column (test_date, session_start_time, or timestamp)")
            continue
        
        print(f"\nProcessing {table_name}...")
        
        # Extract unique combinations
        for idx, row in df.iterrows():
            user_id = str(row[user_id_col]).strip()
            boyfriend_name = str(row[boyfriend_name_col]).strip()
            test_date = normalize_date(row[date_col])
            
            if not user_id or not boyfriend_name or not test_date:
                print(f"[WARNING] Row {idx} in {table_name} has missing values, skipping")
                continue
            
            key = (user_id, test_date, boyfriend_name)
            
            # Generate session_id if not already in mapping
            if key not in session_mapping:
                session_id = generate_session_id_with_date(user_id, boyfriend_name, test_date)
                session_mapping[key] = session_id
                print(f"  Created mapping: ({user_id[:8]}..., {test_date[:10]}..., {boyfriend_name}) -> {session_id}")
    
    print(f"\n[OK] Created {len(session_mapping)} unique session mappings")
    return session_mapping


In [ ]:
def update_session_ids(data: dict, session_mapping: dict):
    """Update session IDs in all dataframes based on the mapping."""
    print("\n" + "=" * 60)
    print("Updating Session IDs")
    print("=" * 60)
    
    updated_data = {}
    
    for table_name, df in data.items():
        if df.empty:
            updated_data[table_name] = df
            continue
        
        print(f"\nUpdating {table_name}...")
        df = df.copy()
        
        # Get required columns - check with case-insensitive matching
        user_id_col = None
        boyfriend_name_col = None
        
        for col in df.columns:
            col_lower = col.lower().strip()
            if col_lower == 'user_id':
                user_id_col = col
            elif col_lower == 'boyfriend_name':
                boyfriend_name_col = col
        
        if not user_id_col or not boyfriend_name_col:
            print(f"[WARNING] {table_name} missing required columns, skipping. Available: {list(df.columns[:10])}")
            updated_data[table_name] = df
            continue
        
        # Get test_date column
        date_col = get_test_date_column(df)
        if not date_col:
            print(f"[WARNING] {table_name} has no date column, skipping")
            updated_data[table_name] = df
            continue
        
        # Update IDs
        updated_count = 0
        skipped_count = 0
        
        for idx, row in df.iterrows():
            user_id = str(row[user_id_col]).strip()
            boyfriend_name = str(row[boyfriend_name_col]).strip()
            test_date = normalize_date(row[date_col])
            
            if not user_id or not boyfriend_name or not test_date:
                skipped_count += 1
                continue
            
            key = (user_id, test_date, boyfriend_name)
            
            if key in session_mapping:
                new_id = session_mapping[key]
                old_id = row.get('id')
                df.at[idx, 'id'] = new_id
                if old_id != new_id:
                    updated_count += 1
            else:
                skipped_count += 1
        
        print(f"  Updated: {updated_count} records")
        if skipped_count > 0:
            print(f"  Skipped: {skipped_count} records (missing values)")
        
        updated_data[table_name] = df
    
    return updated_data


In [ ]:
def save_updated_csvs(updated_data: dict, backup: bool = True):
    """Save updated dataframes to CSV files."""
    print("\n" + "=" * 60)
    print("Saving Updated CSV Files")
    print("=" * 60)
    
    if backup:
        # Create backup directory
        backup_dir = DATA_DIR / "backup"
        backup_dir.mkdir(exist_ok=True)
        print(f"[INFO] Backups will be saved to: {backup_dir}")
    
    for table_name, df in updated_data.items():
        if table_name not in CSV_FILES:
            continue
        
        filename = CSV_FILES[table_name]
        file_path = DATA_DIR / filename
        
        # Backup original if requested
        if backup and file_path.exists():
            backup_path = backup_dir / f"{filename}.backup"
            import shutil
            shutil.copy2(file_path, backup_path)
            print(f"[OK] Backed up {filename}")
        
        # Save updated file with consistent separator
        try:
            df.to_csv(file_path, sep=CSV_SEPARATOR, index=False)
            print(f"[OK] Saved {filename} ({len(df)} records)")
        except Exception as e:
            print(f"[ERROR] Failed to save {filename}: {e}")


In [ ]:
def recalculate_summary_sessions(data: dict):
    """Recalculate Summary_Sessions table from session_responses."""
    print("\n" + "=" * 60)
    print("Recalculating Summary_Sessions")
    print("=" * 60)
    
    if "session_responses" not in data:
        print("[ERROR] session_responses not found in data")
        return None
    
    df = data["session_responses"]
    
    if df.empty:
        print("[WARNING] session_responses is empty")
        return None
    
    # Required columns
    required_cols = ['toxic_score', 'filter_violations']
    if not all(col in df.columns for col in required_cols):
        print(f"[ERROR] session_responses missing required columns: {required_cols}")
        return None
    
    # Convert to numeric
    df['toxic_score'] = pd.to_numeric(df['toxic_score'], errors='coerce')
    df['filter_violations'] = pd.to_numeric(df['filter_violations'], errors='coerce')
    
    # Remove rows with NaN values
    df_clean = df.dropna(subset=['toxic_score', 'filter_violations'])
    
    if df_clean.empty:
        print("[WARNING] No valid records after cleaning")
        return None
    
    # Calculate statistics
    sum_toxic_score = float(df_clean['toxic_score'].sum())
    max_toxic_score = float(df_clean['toxic_score'].max())
    min_toxic_score = float(df_clean['toxic_score'].min())
    avg_toxic_score = float(df_clean['toxic_score'].mean())
    
    sum_filter_violations = int(df_clean['filter_violations'].sum())
    avg_filter_violations = float(df_clean['filter_violations'].mean())
    
    # Count unique sessions (unique id values)
    count_guys = df_clean['id'].nunique()
    
    # Get max IDs from each table (for backward compatibility, but set to 0 as per code)
    max_id_session_responses = 0
    max_id_gtk_responses = 0
    max_id_feedback = 0
    max_id_session_toxicity_rating = 0
    
    last_update_date = datetime.now().strftime(DATE_FORMAT)
    
    summary_data = {
        'summary_id': 1,
        'sum_toxic_score': sum_toxic_score,
        'max_toxic_score': max_toxic_score,
        'min_toxic_score': min_toxic_score,
        'avg_toxic_score': avg_toxic_score,
        'sum_filter_violations': sum_filter_violations,
        'avg_filter_violations': avg_filter_violations,
        'count_guys': count_guys,
        'max_id_session_responses': max_id_session_responses,
        'max_id_gtk_responses': max_id_gtk_responses,
        'max_id_feedback': max_id_feedback,
        'max_id_session_toxicity_rating': max_id_session_toxicity_rating,
        'last_update_date': last_update_date,
    }
    
    print(f"[OK] Calculated Summary_Sessions:")
    print(f"  count_guys: {count_guys}")
    print(f"  avg_toxic_score: {avg_toxic_score:.6f}")
    print(f"  sum_toxic_score: {sum_toxic_score:.6f}")
    print(f"  sum_filter_violations: {sum_filter_violations}")
    
    return summary_data


In [ ]:
def save_summary_sessions(summary_data: dict):
    """Save Summary_Sessions to CSV."""
    if summary_data is None:
        return
    
    print("\n" + "=" * 60)
    print("Saving Summary_Sessions")
    print("=" * 60)
    
    file_path = DATA_DIR / "Summary_Sessions.csv"
    
    # Backup if exists
    if file_path.exists():
        backup_dir = DATA_DIR / "backup"
        backup_dir.mkdir(exist_ok=True)
        import shutil
        backup_path = backup_dir / "Summary_Sessions.csv.backup"
        shutil.copy2(file_path, backup_path)
        print(f"[OK] Backed up Summary_Sessions.csv")
    
    # Create DataFrame and save with consistent separator
    df = pd.DataFrame([summary_data])
    df.to_csv(file_path, sep=CSV_SEPARATOR, index=False)
    print(f"[OK] Saved Summary_Sessions.csv")


In [ ]:
def show_changes_summary(data: dict, updated_data: dict, session_mapping: dict):
    """Show a summary of changes before saving."""
    print("\n" + "=" * 60)
    print("Changes Summary")
    print("=" * 60)
    
    print(f"\nTotal unique sessions: {len(session_mapping)}")
    
    print("\nSession ID changes per table:")
    for table_name in CSV_FILES.keys():
        if table_name not in data or table_name not in updated_data:
            continue
        
        old_df = data[table_name]
        new_df = updated_data[table_name]
        
        if old_df.empty or new_df.empty:
            continue
        
        # Count changed IDs
        if 'id' in old_df.columns and 'id' in new_df.columns:
            changes = (old_df['id'] != new_df['id']).sum()
            total = len(old_df)
            print(f"  {table_name}: {changes}/{total} records updated")
    
    print("\n" + "=" * 60)

def reassign_session_ids_and_recalculate(dry_run: bool = False):
    """
    Main function to reassign session IDs and recalculate Summary_Sessions.
    
    Args:
        dry_run: If True, only show what would be changed without saving files
    """
    print("=" * 60)
    print("Reassign Session IDs and Recalculate Summary_Sessions")
    if dry_run:
        print("DRY RUN MODE - No files will be modified")
    print("=" * 60)
    
    # Step 1: Load data
    data = load_and_prepare_data()
    
    if not data:
        print("[ERROR] No data loaded")
        return
    
    # Step 2: Create session mapping
    session_mapping = create_session_mapping(data)
    
    if not session_mapping:
        print("[ERROR] No session mappings created")
        return
    
    # Step 3: Update session IDs
    updated_data = update_session_ids(data, session_mapping)
    
    # Step 4: Show summary
    show_changes_summary(data, updated_data, session_mapping)
    
    if dry_run:
        print("\n[INFO] Dry run completed. No files were modified.")
        print("Set dry_run=False to apply changes.")
        return
    
    # Step 5: Save updated CSVs
    save_updated_csvs(updated_data, backup=True)
    
    # Step 6: Recalculate Summary_Sessions
    summary_data = recalculate_summary_sessions(updated_data)
    
    # Step 7: Save Summary_Sessions
    save_summary_sessions(summary_data)
    
    print("\n" + "=" * 60)
    print("[SUCCESS] All operations completed!")
    print("=" * 60)


In [ ]:
# Run the reassignment process
# First run with dry_run=True to preview changes, then set to False to apply
# reassign_session_ids_and_recalculate(dry_run=True)  # Preview changes
# reassign_session_ids_and_recalculate(dry_run=False)  # Apply changes
